In [16]:
pip install tiktoken chromadb

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b8/eb/234646d9eefda8a500d0fd88b05bf625a90ed18054124349db26e558276e/tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 503.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/759.8 kB ? eta -:--:--
   --- ------------------------------------ 61.4/759.8 kB 1.7 MB/s eta 0:00:01
   ----------------- ---------------------- 327.7/759.8 kB 4.1 MB/s eta 0:00:01
   ------------------------------------ --- 686.1/759.8 kB 5.4 MB/s eta 0:00:01
   --------------------------

In [1]:
# you need openai key
import os
os.environ['OPENAI_API_KEY'] = ''

In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.schema import StrOutputParser

In [13]:
category = """
[Environmental, Education, Health Care, Criminal Justice, Taxation, Transportation, 
Housing, Civil Rights, Labor and Employment, Budget and Appropriations, Public Safety, Technology and Innovation, 
Immigration, Economic Development, Social Services]
"""

In [23]:
tagging = """
[Climate Change Mitigation,
Renewable Energy Initiatives,
Biodiversity Conservation,
Water Pollution Control,
Sustainable Agriculture Practices,
Air Quality Improvement,
Waste Reduction Programs,
Coastal Erosion Management,
Environmental Impact Assessments,
Wildlife Habitat Protection,
Curriculum Development,
Digital Learning Resources,
Teacher Professional Development,
Early Childhood Education,
Special Education Services,
Literacy Programs,
Vocational Training,
Education Technology Integration,
School Infrastructure Upgrades,
Student Mental Health Support,
Universal Health Coverage,
Healthcare Access for All,
Mental Health Services Expansion,
Disease Prevention Programs,
Elderly Care Services,
Healthcare Quality Standards,
Health Information Technology,
Maternal and Child Health,
Public Health Emergency Preparedness,
Healthcare Workforce Training,
Police Reforms
Prisoner Rehabilitation
Community Policing Initiatives
Restorative Justice Programs
Criminal Sentencing Reform
Victim Support Services
Legal Aid for the Underprivileged
Juvenile Justice System Overhaul
Hate Crime Prevention Measures
Court System Modernization,
Progressive Tax Reform
Corporate Taxation Policies
Income Tax Deductions
Property Tax Relief
Sales Tax Revision
Tax Compliance Regulations
Tax Transparency Measures
Wealth Redistribution Initiatives
Small Business Tax Breaks
Tax Fraud Prevention Measures,
Public Transit Expansion
Road Infrastructure Maintenance
Bike and Pedestrian Path Development
Traffic Congestion Reduction
Electric Vehicle Adoption Incentives
Freight Transportation Efficiency
Aviation Safety Regulations
Railroad Infrastructure Modernization
Intermodal Transportation Integration
Autonomous Vehicle Regulations,
Affordable Housing Development
Homelessness Prevention Programs
Rent Control Measures
Fair Housing Enforcement
Housing Discrimination Prevention
Housing Voucher Program Expansion
Sustainable Housing Initiatives
Urban Redevelopment Plans
Eviction Moratoriums
Homeownership Support Programs,
Anti-Discrimination Laws
Gender Equality Protections
LGBTQ+ Rights Advocacy
Disability Rights Enforcement
Indigenous Peoples' Rights
Religious Freedom Protection
Language Access Policies
Voting Rights Expansion
Equal Pay Legislation
Minority Rights Safeguards,
Minimum Wage Increase
Occupational Safety Regulations
Labor Union Protections
Workplace Harassment Prevention
Job Training and Apprenticeships
Workforce Diversity Initiatives
Employee Benefits Expansion
Fair Working Hours Regulations
Unemployment Benefits Enhancement
Remote Work Policies,
Government Spending Oversight
Emergency Fund Allocation
Public Debt Management
Fiscal Responsibility Audits
Public Infrastructure Investment
Social Welfare Program Funding
Pension Plan Reform
Local Government Grant Programs
Financial Aid for Disadvantaged Communities
Tax Revenue Allocation,
Emergency Response Planning
Disaster Preparedness Training
Cybersecurity Protocols
Domestic Violence Prevention
Fire Safety Regulations
Gun Control Measures
Community Health and Safety Programs
Public Health Crisis Management
Hate Crime Reporting Systems
Crime Prevention Initiatives,
Digital Privacy Laws
Data Security Measures
Innovation Investment Policies
Broadband Infrastructure Expansion
E-Government Service Enhancements
Technology Education in Schools
Artificial Intelligence Regulations
Privacy Protection for Biometric Data
Blockchain Integration Strategies
Open Data Initiatives,
Immigration Policy Reform
Refugee Resettlement Programs
Asylum Seeker Protections
Migrant Worker Rights
Language Access Services
Family Reunification Initiatives
Pathways to Citizenship
Humanitarian Aid for Migrants
Border Security Measures
Anti-Trafficking Efforts,
Small Business Support Programs
Export Promotion Policies
Rural Development Initiatives
Entrepreneurship Training
Trade Agreement Negotiations
Tourism Industry Growth
Regional Economic Integration
Financial Inclusion Programs
Economic Diversification Strategies
Infrastructure Investment Plans,
Child Welfare Services
Domestic Violence Support
Elderly Care Programs
Foster Care System Reforms
Disability Assistance Programs
Community Support Services
Youth Mentorship Programs
Substance Abuse Rehabilitation
Home Care for the Disabled
Affordable Childcare Services
]
"""

In [21]:
# LLM
prompt_template = """Use the context below to assign a category that is relevant to the category, which is delimited by ####
Also, I want you to use the tagging, which is delimited by ****, and give top 3 tags that fit the context.

Context: {context}
Category: {category}
tagging: {tags}
Do not make up false information
"""

In [26]:
info = """
SECTION 1. Chapter 20 Section 23 of the General Laws, as appearing in the 2020 Official Edition, is hereby amended by inserting the following after section (f): -
	(g) Notwithstanding any general or special law to the contrary, the department of agricultural resources, 
    with the approval of the co-holder, if any, in its sole discretion, may allow for storm water mitigation or nitrogen mitigation purposes, 
    in nitrogen sensitive embayment watersheds, uncultivated, planted or other storm water mitigation infrastructure within 200 feet of any resource area in any parcel 
    that has been accepted into the Massachusetts Agricultural Preservation Restriction program."""


PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "category", 'tags'])

with get_openai_callback() as cb:
    llm = LLMChain(
            llm = ChatOpenAI(openai_api_key='',
                     temperature=0.01, model="gpt-3.5-turbo"), prompt=PROMPT)
        
    response = llm.predict(context=info, category=category, tags=tagging)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    
print(info)
print(response)

Total Tokens: 1069
Prompt Tokens: 1040
Completion Tokens: 29
Total Cost (USD): $0.0016180000000000003

SECTION 1. Chapter 20 Section 23 of the General Laws, as appearing in the 2020 Official Edition, is hereby amended by inserting the following after section (f): -
	(g) Notwithstanding any general or special law to the contrary, the department of agricultural resources, 
    with the approval of the co-holder, if any, in its sole discretion, may allow for storm water mitigation or nitrogen mitigation purposes, 
    in nitrogen sensitive embayment watersheds, uncultivated, planted or other storm water mitigation infrastructure within 200 feet of any resource area in any parcel 
    that has been accepted into the Massachusetts Agricultural Preservation Restriction program.
Category: Environmental

Top 3 tags: 
1. Water Pollution Control
2. Sustainable Agriculture Practices
3. Coastal Erosion Management
